In [1]:

import warnings 
import pandas as pd
import numpy as np

warnings.filterwarnings('ignore')
all_games_df = pd.read_csv("/home/rohanbendapudi/github_repos/nba_predictor_development/models/data/all_games.csv")
all_games_df.drop("Unnamed: 0", axis=1, inplace=True)
all_games_df.sort_values(by="GAME_DATE", inplace=True)
all_games_df["WL_A"].replace("W",1,inplace=True)
all_games_df["WL_A"].fillna(0,inplace=True)
all_games_df["WL_A"].replace("L",0,inplace=True)
all_games_df.reset_index(inplace=True,drop=True)

In [8]:
all_games_df.sort_values(by="GAME_DATE", inplace=True)
all_games_df

,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,...,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
0,21983,1610612755,PHL,Philadelphia 76ers,28300003,1983-10-28,PHL vs. WAS,1,240,117,...,0.759,10.0,26.0,36.0,32,9.0,3,21,35,NaN
1,21983,1610612757,POR,Portland Trail Blazers,28300007,1983-10-28,POR vs. SDC,1,240,107,...,0.750,7.0,24.0,31.0,19,5.0,5,18,31,NaN
2,21983,1610612739,CLE,Cleveland Cavaliers,28300008,1983-10-28,CLE vs. NYK,0,240,106,...,0.657,16.0,19.0,35.0,26,9.0,3,17,33,NaN
3,21983,1610612751,NJN,New Jersey Nets,28300005,1983-10-28,NJN vs. ATL,1,240,126,...,0.684,15.0,18.0,33.0,14,10.0,5,17,30,NaN
4,21983,1610612743,DEN,Denver Nuggets,28300009,1983-10-28,DEN vs. UTH,1,240,139,...,0.781,10.0,27.0,37.0,32,6.0,8,7,30,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52802,42023,1610612738,BOS,Boston Celtics,42300401,2024-06-06,BOS vs. DAL,1,241,107,...,0.632,10.0,33.0,43.0,9,8.0,1,11,16,-18.0
52803,42023,1610612738,BOS,Boston Celtics,42300402,2024-06-09,BOS vs. DAL,1,239,105,...,0.667,9.0,34.0,43.0,21,5.0,3,15,17,-7.0
52804,42023,1610612742,DAL,Dallas Mavericks,42300403,2024-06-12,DAL vs. BOS,0,240,99,...,0.929,6.0,30.0,36.0,26,4.0,6,9,19,7.0
52805,42023,1610612742,DAL,Dallas Mavericks,42300404,2024-06-14,DAL vs. BOS,1,240,122,...,0.923,4.0,27.0,31.0,18,2.0,5,13,19,-38.0


In [2]:
columns = ["FG_PCT", "FG3_PCT", "FTM", "OREB", "DREB", "REB", "AST"]
columns_a = [column + "_A" for column in columns]
columns_b = [column + "_B" for column in columns]

In [3]:
from api_helpers.team_stats_helpers import find_team_id

team_ids = [team_id[0] for team_id in find_team_id()]

In [4]:
all_games_df["AST_A"].rolling(window=5).mean().to_frame()

,AST_A
0,NaN
1,NaN
2,NaN
3,NaN
4,28.6
...,...
52802,21.2
52803,23.2
52804,21.4
52805,21.0


In [5]:
# rolling average for team a
a_avg_df = pd.DataFrame()
string_columns = ["GAME_ID", "TEAM_ID_A", "TEAM_ID_B", "GAME_ID"]


for team_id in team_ids:
    team_a_df = all_games_df[all_games_df["TEAM_ID_A"] == team_id]
    team_a_mean = team_a_df[columns_a].rolling(window=5).mean()

    local_a_df = pd.concat([team_a_df[string_columns], team_a_mean], axis=1)

    a_avg_df = pd.concat([a_avg_df, local_a_df])


a_avg_df

,GAME_ID,TEAM_ID_A,TEAM_ID_B,GAME_ID,FG_PCT_A,FG3_PCT_A,FTM_A,OREB_A,DREB_A,REB_A,AST_A
18,28300014,1610612737,1610612765,28300014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,28300027,1610612737,1610612764,28300027,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,28300041,1610612737,1610612741,28300041,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,28300101,1610612737,1610612746,28300101,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,28300112,1610612737,1610612755,28300112,0.5088,NaN,20.2,17.0,27.2,44.2,24.6
...,...,...,...,...,...,...,...,...,...,...,...
52605,22301087,1610612766,1610612738,22301087,0.4614,0.3220,14.0,9.0,30.6,39.6,26.6
52621,22301102,1610612766,1610612757,22301102,0.4690,0.3218,13.4,7.0,30.2,37.2,27.0
52643,22301115,1610612766,1610612753,22301115,0.4726,0.3430,14.6,6.2,30.6,36.8,26.8
52656,22301135,1610612766,1610612760,22301135,0.5000,0.3758,13.4,5.2,30.6,35.8,28.0


In [ ]:
b_avg_df = pd.DataFrame()

for team_id in team_ids:
    team_b_df = all_games_df[all_games_df["TEAM_ID_B"] == team_id]
    team_b_mean = team_b_df[columns_b].rolling(window=5).mean()

    # local_b_df = pd.concat([team_b_df,team_b_mean], axis=1)

    b_avg_df = pd.concat([b_avg_df,team_b_mean])
    

pd.concat([a_avg_df,b_avg_df],axis=1)

,GAME_ID,TEAM_ID_A,TEAM_ID_B,GAME_ID,FG_PCT_A,FG3_PCT_A,FTM_A,OREB_A,DREB_A,REB_A,AST_A,FG_PCT_B,FG3_PCT_B,FTM_B,OREB_B,DREB_B,REB_B,AST_B
18,28300014,1610612737,1610612765,28300014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,28300027,1610612737,1610612764,28300027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,28300041,1610612737,1610612741,28300041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,28300101,1610612737,1610612746,28300101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.4822,NaN,20.4,17.4,24.6,42.0,25.4
116,28300112,1610612737,1610612755,28300112,0.5088,NaN,20.2,17.0,27.2,44.2,24.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52605,22301087,1610612766,1610612738,22301087,0.4614,0.3220,14.0,9.0,30.6,39.6,26.6,0.4890,0.3758,14.0,10.4,34.4,44.8,26.4
52621,22301102,1610612766,1610612757,22301102,0.4690,0.3218,13.4,7.0,30.2,37.2,27.0,0.4328,0.2910,9.2,10.2,33.6,43.8,23.2
52643,22301115,1610612766,1610612753,22301115,0.4726,0.3430,14.6,6.2,30.6,36.8,26.8,0.4690,0.3576,19.2,9.8,29.0,38.8,21.8
52656,22301135,1610612766,1610612760,22301135,0.5000,0.3758,13.4,5.2,30.6,35.8,28.0,0.4798,0.3380,17.6,9.0,29.4,38.4,27.4


In [ ]:
from api_helpers.team_stats_helpers import find_team_id


def avg_last_n_games(all_games_df: pd.DataFrame, columns: list, team="TEAM_ID_A", n=5):

    avg_df = pd.DataFrame()

    team_ids = [team_id[0] for team_id in find_team_id()]

    for team_id in team_ids:
        team_df = all_games_df[all_games_df[team] == team_id]
        team_mean = team_df[columns].rolling(window=5).mean()

        avg_df = pd.concat([avg_df, team_mean])
        
    return avg_df

In [ ]:
def load_past_n_games(
    all_games_df: pd.DataFrame,
    columns: list = ["FG_PCT", "FG3_PCT", "FTM", "OREB", "DREB", "REB", "AST"],
    n=5,
):
    string_columns = ["GAME_ID", "TEAM_ID_A", "TEAM_ID_B", "GAME_ID", "WL_A"]
    columns_a = [column + "_A" for column in columns]
    columns_b = [column + "_B" for column in columns]

    a_avg_df = avg_last_n_games(all_games_df, columns_a, team="TEAM_ID_A", n=n)
    b_avg_df = avg_last_n_games(all_games_df, columns_b, team="TEAM_ID_B", n=n)

    last_n_df = pd.concat([all_games_df[string_columns], a_avg_df, b_avg_df], axis=1)

    return last_n_df


load_past_n_games(all_games_df, columns, 5)

,GAME_ID,TEAM_ID_A,TEAM_ID_B,GAME_ID,WL_A,FG_PCT_A,FG3_PCT_A,FTM_A,OREB_A,DREB_A,REB_A,AST_A,FG_PCT_B,FG3_PCT_B,FTM_B,OREB_B,DREB_B,REB_B,AST_B
0,28300003,1610612755,1610612764,28300003,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28300007,1610612757,1610612746,28300007,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,28300008,1610612739,1610612752,28300008,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,28300005,1610612751,1610612737,28300005,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,28300009,1610612743,1610612762,28300009,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52802,42300401,1610612738,1610612742,42300401,1,0.4846,0.3580,16.8,10.6,30.2,40.8,27.0,0.4946,0.3454,14.0,10.0,35.2,45.2,20.0
52803,42300402,1610612738,1610612742,42300402,1,0.4924,0.3634,16.6,11.4,31.4,42.8,28.4,0.4844,0.3116,15.2,10.0,34.6,44.6,18.8
52804,42300403,1610612742,1610612738,42300403,0,0.4684,0.4048,17.4,10.0,31.4,41.4,21.6,0.4764,0.3586,15.2,8.8,32.8,41.6,21.8
52805,42300404,1610612742,1610612738,42300404,1,0.4876,0.4172,18.0,10.2,32.2,42.4,20.6,0.4464,0.3504,15.4,7.8,31.2,39.0,21.0
